# Manual Deployment of UUPS contracts for ownership by multisig vault

## Motivation
Tools like REMIX IDE and HardHat have plugins that automate and abstract the proxy contract creation and deployment.  This makes it hard to ensure that multisig vaults / wallets / contracts own the deployed assets.


In [1]:
import json
import web3

from web3 import Web3
from solcx import compile_standard, install_solc
from web3.contract import ConciseContract
from pathlib import Path
from crypto_wallet import generate_account, get_balance,send_transaction
w3 = Web3(Web3.HTTPProvider('HTTP://127.0.0.1:7545'))

In [2]:
def get_contract_source(file_name):
    with open(file_name) as f:
        return f.read()

In [3]:
# py-solc-x documentation at https://solcx.readthedocs.io/en/latest/using-the-compiler.html
# good tutorial at https://coinsbench.com/how-to-compile-solidity-smart-contracts-with-python-98249cfc0879
# Install Solidity compiler.
_solc_version = "0.8.9"
install_solc(_solc_version)

Version('0.8.9')

In [4]:
oz_strings =  get_contract_source(Path('Contracts/openzeppelin/utils/Strings.sol'))
oz_context =  get_contract_source(Path('Contracts/openzeppelin/utils/Context.sol'))
oz_introspection_ierc165 =  get_contract_source(Path('Contracts/openzeppelin/utils/introspection/IERC165.sol'))
oz_introspection_erc165 =  get_contract_source(Path('Contracts/openzeppelin/utils/introspection/ERC165.sol'))
oz_i_access_control = get_contract_source(Path('Contracts/openzeppelin/access/IAccessControl.sol'))
oz_access_control = get_contract_source(Path('Contracts/openzeppelin/access/AccessControl.sol'))
oz_address = get_contract_source(Path('Contracts/openzeppelin/utils/Address.sol'))
oz_initializable = get_contract_source(Path('Contracts/openzeppelin/proxy/utils/Initializable.sol'))
oz_uups_upgradeable = get_contract_source(Path('Contracts/openzeppelin/proxy/utils/UUPSUpgradeable.sol'))
oz_ierc1822 = get_contract_source(Path('Contracts/openzeppelin/interfaces/draft-IERC1822.sol'))
oz_erc1967upgrade = get_contract_source(Path('Contracts/openzeppelin/proxy/ERC1967/ERC1967Upgrade.sol'))
oz_beacon_ibeacon = get_contract_source(Path('Contracts/openzeppelin/proxy/beacon/IBeacon.sol'))
oz_storage_slot = get_contract_source(Path('Contracts/openzeppelin/utils/StorageSlot.sol'))
oz_erc1697_proxy = get_contract_source(Path('Contracts/openzeppelin/proxy/ERC1967/ERC1967Proxy.sol'))
oz_erc1697_upgrade_proxy = get_contract_source(Path('Contracts/openzeppelin/proxy/ERC1967/ERC1967Upgrade.sol'))
oz_proxy = get_contract_source(Path('Contracts/openzeppelin/proxy/Proxy.sol'))

sabot_staking_proxy = get_contract_source(Path('Contracts/SabotStakingProxy.sol'))
sabot_staking = get_contract_source(Path('Contracts/SabotStakingV1.sol'))

In [5]:
# compile the logic contract
# Compile SimpleStorage smart contract with solcx.


compiled_contracts = compile_standard(
    {
        "language": "Solidity",
        "sources": {
            'openzeppelin/utils/Strings.sol':{"content": oz_strings},
            'openzeppelin/utils/Context.sol':{"content": oz_context},
            'openzeppelin/utils/introspection/IERC165.sol':{"content": oz_introspection_ierc165},
            'openzeppelin/utils/introspection/ERC165.sol':{"content": oz_introspection_erc165},
            'openzeppelin/access/IAccessControl.sol':{"content": oz_i_access_control},
            'openzeppelin/access/AccessControl.sol':{"content": oz_access_control},
            'openzeppelin/utils/Address.sol':{"content": oz_address},
            'openzeppelin/proxy/utils/Initializable.sol':{"content": oz_initializable},
            'openzeppelin/interfaces/draft-IERC1822.sol':{"content": oz_ierc1822},
            'openzeppelin/proxy/beacon/IBeacon.sol':{"content": oz_beacon_ibeacon},
            'openzeppelin/utils/StorageSlot.sol':{"content": oz_storage_slot},
            'openzeppelin/proxy/ERC1967/ERC1967Upgrade.sol':{"content": oz_erc1967upgrade},
            'openzeppelin/proxy/utils/UUPSUpgradeable.sol':{"content": oz_uups_upgradeable},
            'SabotStakingV1.sol': {"content": sabot_staking},
            'openzeppelin/proxy/Proxy.sol':{"content": oz_proxy},
            'openzeppelin/proxy/ERC1967/ERC1967Upgrade.sol':{"content": oz_erc1697_upgrade_proxy},
            'openzeppelin/proxy/ERC1967/ERC1967Proxy.sol':{"content": oz_erc1697_proxy},
            'SabotStakingProxy.sol': {"content": sabot_staking_proxy}
            
        },
        "settings": {
            "outputSelection": {
                "*": {"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]}
            }
        },
    },
    solc_version=_solc_version,
)


In [6]:
account = generate_account()
account.address

'0x86C7A44De7bF114E5bA5363f69C24c2E8C29318d'

In [7]:
compiled_contracts['contracts']['SabotStakingV1.sol']['SabotStakingV1'].keys()

dict_keys(['abi', 'evm', 'metadata'])

In [8]:
compiled_contracts['contracts']['SabotStakingV1.sol']['SabotStakingV1']['evm']['bytecode']['object']

'...'

In [9]:
compiled_contracts['contracts']['SabotStakingV1.sol']['SabotStakingV1']['evm']['bytecode'].keys()

dict_keys(['functionDebugData', 'generatedSources', 'linkReferences', 'object', 'opcodes', 'sourceMap'])

In [10]:
SabotStakingV1 = w3.eth.contract(
    abi=compiled_contracts['contracts']['SabotStakingV1.sol']['SabotStakingV1']['abi'], 
    bytecode=compiled_contracts['contracts']['SabotStakingV1.sol']['SabotStakingV1']['evm']['bytecode']['object'])


In [11]:
# Submit the transaction that deploys the contract
tx_hash = SabotStakingV1.constructor().transact({'from': account.address , 'gas': 3000000})
print ("Tx submitted: ", w3.toHex(tx_hash))  # added by me.

Tx submitted:  0xd3aa2b1a383b364f9e5e4aec48590698bc9f5a7cbbf1f487b070849ed47bf65c


In [12]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [13]:
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xd3aa2b1a383b364f9e5e4aec48590698bc9f5a7cbbf1f487b070849ed47bf65c'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x668e2db50b3a42185d861f1f412e6eae17025ab00c7842ae0e58e459259255aa'),
 'blockNumber': 30,
 'from': '0x86C7A44De7bF114E5bA5363f69C24c2E8C29318d',
 'to': None,
 'gasUsed': 2830083,
 'cumulativeGasUsed': 2830083,
 'contractAddress': '0xBD00f460c0Ec48310D10fb4EBEdf711BAC15f661',
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xd3aa2b1a383b364f9e5e4aec48590698bc9f5a7cbbf1f487b070849ed47bf65c'),
   'blockHash': HexBytes('0x668e2db50b3a42185d861f1f412e6eae17025ab00c7842ae0e58e459259255aa'),
   'blockNumber': 30,
   'address': '0xBD00f460c0Ec48310D10fb4EBEdf711BAC15f661',
   'data': '0x',
   'topics': [HexBytes('0x2f8788117e7eff1d82e926ec794901d17c78024a50270940304540a733656f0d'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x00000

In [14]:
logic_contract_address=tx_receipt.contractAddress
logic_contract_address

'0xBD00f460c0Ec48310D10fb4EBEdf711BAC15f661'

In [15]:
SabotStakingV1_instance = w3.eth.contract(
    address=logic_contract_address,
    abi=compiled_contracts['contracts']['SabotStakingV1.sol']['SabotStakingV1']['abi'],
)

In [16]:
SabotStakingV1_instance.functions.balance().call()

0

In [17]:
sellToken='0x04324f495a27fa08d5E8CC3F01178F55C46D53bf'
sellTokenAmount=777770000000
buyToken='0x6e8d8c332fAF8b71E312F5377D9dd94baDE0d744'

In [18]:
tx_hash = SabotStakingV1_instance.functions.swap(sellToken, sellTokenAmount, buyToken).transact({'from': account.address , 'gas': 3000000})
receipt = w3.eth.getTransactionReceipt(tx_hash)
logs = SabotStakingV1_instance.events.SabotSwap().processReceipt(receipt)
logs

(AttributeDict({'args': AttributeDict({'sellToken': '0x04324f495a27fa08d5E8CC3F01178F55C46D53bf',
   'sellTokenAmount': 777770000000,
   'buyToken': '0x6e8d8c332fAF8b71E312F5377D9dd94baDE0d744',
   'buyTokenAmount': 42}),
  'event': 'SabotSwap',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x023e8a654572e5b0ef39e380df7d3bd77c90543dd183e77478e659b3a646cacc'),
  'address': '0xBD00f460c0Ec48310D10fb4EBEdf711BAC15f661',
  'blockHash': HexBytes('0xd3af163aed0819028a29d094c60271edd5f8d656d5b4a21c00ea6c46109840cd'),
  'blockNumber': 31}),)

### Logic contract deployed and verified

## Proxy Contract

In [19]:
SabotStakingProxy = w3.eth.contract(
    abi=compiled_contracts['contracts']['SabotStakingProxy.sol']['SabotStakingProxy']['abi'], 
    bytecode=compiled_contracts['contracts']['SabotStakingProxy.sol']['SabotStakingProxy']['evm']['bytecode']['object'])


In [20]:
# Submit the transaction that deploys the contract
tx_hash = SabotStakingProxy.constructor(logic_contract_address,bytes()).transact({'from': account.address , 'gas': 3000000})
print ("Tx submitted: ", w3.toHex(tx_hash))  # added by me.

Tx submitted:  0xc68b5ae06c2432e2c20c3f3d9d076ea2d89c9e50c797d16a4fe6a77a59311ed9


In [21]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [22]:
proxy_contract_address=tx_receipt.contractAddress
proxy_contract_address

'0x3830a870dE6b28AF2999A8A6944e9D6042A82861'

In [23]:
proxy_instance = w3.eth.contract(
    address=proxy_contract_address,
    abi=compiled_contracts['contracts']['SabotStakingV1.sol']['SabotStakingV1']['abi'],
)

In [24]:
tx_hash = proxy_instance.functions.initialize().transact({'from': account.address , 'gas': 3000000})
receipt = w3.eth.getTransactionReceipt(tx_hash)
receipt

AttributeDict({'transactionHash': HexBytes('0xe637a45fb76a8be750524ecb98f13fc91441b75b65185ba9b8fd58c632c0db02'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x4eded15f87609364a3a7f502b2ac701215918913d502219313416b3330352a00'),
 'blockNumber': 33,
 'from': '0x86C7A44De7bF114E5bA5363f69C24c2E8C29318d',
 'to': '0x3830a870dE6b28AF2999A8A6944e9D6042A82861',
 'gasUsed': 147187,
 'cumulativeGasUsed': 147187,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xe637a45fb76a8be750524ecb98f13fc91441b75b65185ba9b8fd58c632c0db02'),
   'blockHash': HexBytes('0x4eded15f87609364a3a7f502b2ac701215918913d502219313416b3330352a00'),
   'blockNumber': 33,
   'address': '0x3830a870dE6b28AF2999A8A6944e9D6042A82861',
   'data': '0x',
   'topics': [HexBytes('0x2f8788117e7eff1d82e926ec794901d17c78024a50270940304540a733656f0d'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x0000000

In [29]:
proxy_instance.functions.balance().call()

84

In [44]:
tx_hash = proxy_instance.functions.swap(sellToken, sellTokenAmount, buyToken).transact({'from': account.address , 'gas': 3000000})
receipt = w3.eth.getTransactionReceipt(tx_hash)
logs = proxy_instance.events.SabotSwap().processReceipt(receipt)
logs

(AttributeDict({'args': AttributeDict({'sellToken': '0x04324f495a27fa08d5E8CC3F01178F55C46D53bf',
   'sellTokenAmount': 777770000000,
   'buyToken': '0x6e8d8c332fAF8b71E312F5377D9dd94baDE0d744',
   'buyTokenAmount': 42}),
  'event': 'SabotSwap',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x206f7792c7bb86b089c5975de79772b75637168302fa73f5c79f62d057aa446a'),
  'address': '0x3830a870dE6b28AF2999A8A6944e9D6042A82861',
  'blockHash': HexBytes('0xe5c5615e2394ad94e6592aeaaa20a11651168048631e6a7bf62e8447d71e22f9'),
  'blockNumber': 50}),)